# Data Science / Machine Learning Meetup #1 Deep Learning Hands-on
# 公開データの取得と変換

## はじめに

演習の概略は以下の通りです。
1. [環境準備](#環境準備)
1. [WEBスクレイピング](#WEBスクレイピング)
1. [HiveによるJsonデータ変換](#HiveによるJsonデータ変換)


以下の点にご注意ください。
- 実行するコードの中に、ご利用中のユーザー名に合わせて、変更していただく部分があります。

## 環境準備

### パッケージのインストールとインポート

In [1]:
!pip3 install ipython-sql==0.3.9
!pip3 install PyHive==0.6.1
!pip3 install SQLAlchemy==1.3.13
!pip3 install thrift==0.13.0
!pip3 install sasl==0.2.1
!pip3 install thrift_sasl==0.3.0

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


上記でインストールしたPyHiveは、Pythonコードの中でimportして使われるのではなく、Hiveへの接続の際の接続文字列：`sqlalchemy.create_engine('hive://<host>:<port>')`の中でdialectsとして指定された際に必要になります。そのため、インストール後に利用するためには、新しくプロセスを始める必要があります。**インストールした後に一度、KernelをRestartしてください。**インストールしたプロセスでは、接続時に下記のようなエラーが発生します。
`NoSuchModuleError: Can't load plugin: sqlalchemy.dialects:hive`

In [2]:
import json
import os
import random
import re
import subprocess
import glob
import traceback
from datetime import datetime

from pyhive import hive
import sqlalchemy

import sys

from operator import add
from pyspark.sql import SparkSession

## WEBスクレイピング

無償で利用できるAPIを用いて演習を行います。そのため、利用に一定の制限が課せられることにご留意ください。
例えば、ご利用状況に応じて、下記のようなエラーメッセージを受け取ることがあります。

```
{"response":{"status":429},"errors":[{"message":"Rate limit exceeded. Client may not make more than 200 requests an hour."}]}
```
まず、APIで取得したデータをCDSWプロジェクト内のファイルとして保存します。

取得する銘柄の候補が、`ticker.txt`に定義されています。

In [3]:
ticker_file = open("ticker.txt")
data = ticker_file.readlines()
ticker_file.close()

ticker_list = [i.rstrip('\n') for i in data]

print(len(ticker_list))
print(ticker_list)

2882
['A', 'AA', 'AAL', 'AAN', 'AAOI', 'AAON', 'AAP', 'AAPL', 'AAWW', 'AAXN', 'ABBV', 'ABC', 'ABCB', 'ABEO', 'ABG', 'ABM', 'ABMD', 'ABT', 'ABTX', 'ACA', 'ACAD', 'ACCO', 'ACEL', 'ACGL', 'ACHC', 'ACHN', 'ACHV', 'ACIA', 'ACIW', 'ACLS', 'ACM', 'ACN', 'ACNB', 'ACOR', 'ACRS', 'ACRX', 'ACTG', 'ADBE', 'ADES', 'ADI', 'ADM', 'ADMA', 'ADMP', 'ADMS', 'ADP', 'ADPT', 'ADRO', 'ADS', 'ADSK', 'ADSW', 'ADT', 'ADTN', 'ADUS', 'ADVM', 'ADXS', 'AE', 'AEE', 'AEGN', 'AEIS', 'AEL', 'AEM', 'AEMD', 'AEO', 'AEP', 'AERI', 'AES', 'AFG', 'AFI', 'AFL', 'AG', 'AGCO', 'AGEN', 'AGFS', 'AGI', 'AGIO', 'AGLE', 'AGM', 'AGN', 'AGO', 'AGR', 'AGRX', 'AGS', 'AGTC', 'AGX', 'AGYS', 'AHC', 'AHCO', 'AIG', 'AIMC', 'AIMT', 'AIN', 'AIR', 'AIRG', 'AIRT', 'AIT', 'AIZ', 'AJG', 'AJRD', 'AKAM', 'AKBA', 'AKCA', 'AKRO', 'AKRX', 'AKS', 'AL', 'ALB', 'ALCO', 'ALDX', 'ALE', 'ALEC', 'ALG', 'ALGN', 'ALGT', 'ALIM', 'ALK', 'ALKS', 'ALL', 'ALLK', 'ALLO', 'ALLY', 'ALNY', 'ALOT', 'ALPN', 'ALRM', 'ALRN', 'ALSK', 'ALSN', 'ALT', 'ALTR', 'ALV', 'ALXN', 'AM

In [3]:
!mkdir ./data

In [53]:
symbols = ['BBRY', 'AAPL', 'AMZN', 'BABA', 'YHOO', 'FB', 'GOOG', 'BBBY', 'JNUG', 'SBUX', 'MU']

NUM_REQUEST = 200 - len(symbols)

random.seed(12345)
symbols.extend(random.sample(ticker_list, NUM_REQUEST))

args = ['curl', '-X', 'GET', '']
URL = "https://api.stocktwits.com/api/2/streams/symbol/"

FILE_PATH = "./data/"

start_datetime = datetime.now().strftime("%Y%m%d_%H%M")
for symbol in symbols:
    try:
        args[3] = URL + symbol + ".json"
        print(args[3])
        proc = subprocess.run(args,stdout = subprocess.PIPE, stderr = subprocess.PIPE)

        path = FILE_PATH + symbol + "_" + start_datetime + ".json"
        print(path)
        with open(path, mode='w') as f:
            f.write(proc.stdout.decode("utf8"))
    except:
        traceback.print_exc()

https://api.stocktwits.com/api/2/streams/symbol/BBRY.json
./data/BBRY_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/AAPL.json
./data/AAPL_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/AMZN.json
./data/AMZN_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/BABA.json
./data/BABA_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/YHOO.json
./data/YHOO_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/FB.json
./data/FB_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/GOOG.json
./data/GOOG_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/BBBY.json
./data/BBBY_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/JNUG.json
./data/JNUG_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/SBUX.json
./data/SBUX_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/MU.json
./data/MU_20200212_0838.json
https://api.stocktwits.com/ap

./data/IMMR_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/ADUS.json
./data/ADUS_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/AR.json
./data/AR_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/ATO.json
./data/ATO_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/NRC.json
./data/NRC_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/BCC.json
./data/BCC_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/MATX.json
./data/MATX_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/CZZ.json
./data/CZZ_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/ADS.json
./data/ADS_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/LFUS.json
./data/LFUS_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/ENVA.json
./data/ENVA_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/WIRE.json
./data/WIRE_20200212_0838.json
http

./data/XNCR_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/WAFD.json
./data/WAFD_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/ATH.json
./data/ATH_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/FTR.json
./data/FTR_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/MYOK.json
./data/MYOK_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/AOS.json
./data/AOS_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/LBY.json
./data/LBY_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/PZZA.json
./data/PZZA_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/RLI.json
./data/RLI_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/SMED.json
./data/SMED_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/CAG.json
./data/CAG_20200212_0838.json
https://api.stocktwits.com/api/2/streams/symbol/TRU.json
./data/TRU_20200212_0838.json
http

正常なレスポンス・ステータスを持っていないファイルを取り除きます。

In [54]:
!grep -rlv '{"response":{"status":200}' data
!grep -rlv '{"response":{"status":200}' data | xargs rm

data/REX_20200212_0838.json
data/HAE_20200212_0838.json
data/BBY_20200212_0838.json
data/DDS_20200212_0838.json
data/MRNA_20200212_0838.json
data/LNN_20200212_0838.json
data/TROW_20200212_0838.json
data/PINC_20200212_0838.json
data/IMMR_20200212_0838.json
data/EPAM_20200212_0838.json
data/CREE_20200212_0838.json
data/OFLX_20200212_0838.json
data/XBIT_20200212_0838.json
data/JBL_20200212_0838.json
data/RLI_20200212_0838.json
data/GWB_20200212_0838.json
data/CHMA_20200212_0838.json
data/ADMA_20200212_0838.json
data/HMHC_20200212_0838.json
data/FARM_20200212_0838.json
data/NEOG_20200212_0838.json
data/DGLY_20200212_0838.json
data/FULC_20200212_0838.json
data/BZH_20200212_0838.json
data/INVA_20200212_0838.json
data/CLDX_20200212_0838.json
data/FBIZ_20200212_0838.json
data/WIRE_20200212_0838.json
data/MLND_20200212_0838.json
data/JJSF_20200212_0838.json
data/BRKL_20200212_0838.json
data/BLBD_20200212_0838.json
data/SSRM_20200212_0838.json
data/ENVA_20200212_0838.json
data/CTL_20200212_0838.

次に、保存したファイルを、分散処理環境（クラスター）を使って加工するためにHDFSへコピーします。

**コピー先を、（必要に応じて）自分のフォームディレクトリに置換してください。**
ここでは/tmp/フォルダを利用します。

In [6]:
!export HADOOP_CONF_DIR=/etc/hadoop/conf; hdfs dfs -mkdir /tmp/twits/

mkdir: `/tmp/twits': File exists


In [7]:
!export HADOOP_CONF_DIR=/etc/hadoop/conf; hdfs dfs -put ./data/* /tmp/twits/

In [8]:
!export HADOOP_CONF_DIR=/etc/hadoop/conf; hdfs dfs -ls /tmp/twits

Found 294 items
-rw-r--r--   3 user1 supergroup      46715 2020-01-23 06:43 /tmp/twits/AAPL_20200123_0613.json
-rw-r--r--   3 user1 supergroup      46875 2020-01-23 06:44 /tmp/twits/AAPL_20200123_0618.json
-rw-r--r--   3 user1 supergroup      45616 2020-01-23 07:36 /tmp/twits/AAPL_20200123_0717.json
-rw-r--r--   3 user1 supergroup      45696 2020-01-23 07:35 /tmp/twits/AAPL_20200123_0730.json
-rw-r--r--   3 user1 supergroup        125 2020-01-23 07:35 /tmp/twits/AAPL_20200123_0731.json
-rw-r--r--   3 user1 supergroup        125 2020-01-23 07:37 /tmp/twits/AAPL_20200123_0732.json
-rw-r--r--   3 user1 supergroup        125 2020-01-23 07:36 /tmp/twits/AAPL_20200123_0733.json
-rw-r--r--   3 user1 supergroup        125 2020-01-23 07:35 /tmp/twits/AAPL_20200123_0734.json
-rw-r--r--   3 user1 supergroup      49348 2020-02-12 08:21 /tmp/twits/AAPL_20200212_0818.json
-rw-r--r--   3 user1 supergroup      33499 2020-02-12 08:21 /tmp/twits/ABEO_20200212_0818.json
-rw-r--r--   3 user1 supergroup   

-rw-r--r--   3 user1 supergroup      42776 2020-02-12 08:21 /tmp/twits/CYH_20200212_0818.json
-rw-r--r--   3 user1 supergroup      42617 2020-02-12 08:21 /tmp/twits/CZNC_20200212_0818.json
-rw-r--r--   3 user1 supergroup      42997 2020-02-12 08:21 /tmp/twits/CZZ_20200212_0818.json
-rw-r--r--   3 user1 supergroup      46687 2020-02-12 08:21 /tmp/twits/DBD_20200212_0818.json
-rw-r--r--   3 user1 supergroup      43889 2020-02-12 08:21 /tmp/twits/DBI_20200212_0818.json
-rw-r--r--   3 user1 supergroup      42401 2020-02-12 08:21 /tmp/twits/DDS_20200212_0818.json
-rw-r--r--   3 user1 supergroup      42276 2020-02-12 08:21 /tmp/twits/DELL_20200212_0818.json
-rw-r--r--   3 user1 supergroup      37036 2020-02-12 08:21 /tmp/twits/DFS_20200212_0818.json
-rw-r--r--   3 user1 supergroup      30124 2020-02-12 08:21 /tmp/twits/DGLY_20200212_0818.json
-rw-r--r--   3 user1 supergroup      45476 2020-02-12 08:21 /tmp/twits/DK_20200212_0818.json
-rw-r--r--   3 user1 supergroup      47915 2020-02-12 08:2

## HiveによるJsonデータ変換

クラスターでデータを変換します。CDSW上では、ユーザーごとに別のプロジェクトを使っていましたが、クラスター環境では、自分が利用しているユーザーとデータを意識して取り扱う必要があります。


あなたの（HADOOPクラスターへアクセスする）ユーザ名は以下で確認できます。

In [9]:
!echo $HADOOP_USER_NAME

user1


### データベースの準備



**下記のセルの中を適切なユーザ名とURL（Hiveサーバー）に置換してください。**

In [10]:
%load_ext sql

**下記のセルの中を適切なユーザ名（必要に応じて）とURL（Hiveサーバー）に置換してください。**

In [11]:
%sql hive://user1@master.ykono.work:10000

'Connected: user1@None'

**（必要に応じて）あなたのユーザ名でデータベースを作成・利用してください**

In [ ]:
%sql CREATE DATABASE <your user name>

以下では、defaultデータベースを使います。

In [12]:
%sql USE default
%sql SHOW TABLES

 * hive://user1@master.ykono.work:10000
Done.
 * hive://user1@master.ykono.work:10000
Done.


tab_name
json_message
twits


### ライブラリファイルのコピー・登録

Hiveクエリの中でjsonファイルを扱えるようにするためのライブラリを登録します。
ライブラリファイルはGithubリポジトリに含まれています（ライブラリの詳細は`/lib/README.jar`を参照ください）。
はじめにCDSWからHDFSにコピーし、HDFS上のファイルをHiveへ登録します。

コンパイル済みのライブラリファイルをリポジトリに含めています。
- json-1.3.7.3.jar
- json-serde-cdh5-shim-1.3.7.3.jar
- json-serde-1.3.7.3.jar'

- brickhouse-0.7.1-SNAPSHOT.jar

In [13]:
!export HADOOP_CONF_DIR=/etc/hadoop/conf; hdfs dfs -put `ls -1 ./lib/*.jar` .; hdfs dfs -ls /tmp/*.jar

put: `brickhouse-0.7.1-SNAPSHOT.jar': File exists
put: `json-1.3.7.3.jar': File exists
put: `json-serde-1.3.7.3.jar': File exists
put: `json-serde-cdh5-shim-1.3.7.3.jar': File exists
-rw-r--r--   3 user1 supergroup     308146 2020-01-23 06:42 /tmp/brickhouse-0.7.1-SNAPSHOT.jar
-rw-r--r--   3 user1 supergroup      44477 2020-01-23 06:42 /tmp/json-1.3.7.3.jar
-rw-r--r--   3 user1 supergroup      36653 2020-01-23 06:42 /tmp/json-serde-1.3.7.3.jar
-rw-r--r--   3 user1 supergroup       5110 2020-01-23 06:42 /tmp/json-serde-cdh5-shim-1.3.7.3.jar


**(必要に応じ)下記のパスを適切なユーザ名で置換してください。**

ここでは/tmp/フォルダを利用します。

In [14]:
%sql add jar hdfs:/tmp/json-1.3.7.3.jar
%sql add jar hdfs:/tmp/json-serde-1.3.7.3.jar
%sql add jar hdfs:/tmp/json-serde-cdh5-shim-1.3.7.3.jar
%sql add jar hdfs:/tmp/brickhouse-0.7.1-SNAPSHOT.jar
%sql CREATE TEMPORARY FUNCTION to_json AS 'brickhouse.udf.json.ToJsonUDF'

 * hive://user1@master.ykono.work:10000
Done.
 * hive://user1@master.ykono.work:10000
Done.
 * hive://user1@master.ykono.work:10000
Done.
 * hive://user1@master.ykono.work:10000
Done.
 * hive://user1@master.ykono.work:10000
Done.


[]

In [24]:
%sql DROP TABLE IF EXISTS twits
%sql DROP TABLE IF EXISTS message_extracted
%sql DROP TABLE IF EXISTS message_filtered
%sql DROP TABLE IF EXISTS message_exploded
%sql DROP TABLE IF EXISTS sentiment_data

 * hive://user1@master.ykono.work:10000
Done.
 * hive://user1@master.ykono.work:10000
Done.
 * hive://user1@master.ykono.work:10000
Done.
 * hive://user1@master.ykono.work:10000
Done.
 * hive://user1@master.ykono.work:10000
Done.


[]

SNSメッセージファイルを格納した場所を指定して、テーブルを作成します。

**`LOCATION`指定にあなたがファイルをアップロードしたパスを指定してください**
ここでは/tmp/フォルダを利用します。

In [25]:
%%sql
CREATE EXTERNAL TABLE twits (
	messages 
	ARRAY<
	    STRUCT<body: STRING,
	        symbols:ARRAY<STRUCT<symbol:STRING>>,
	        entities:STRUCT<sentiment:STRUCT<basic:STRING>>
	    >
	>
)
ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe' 
STORED AS TEXTFILE
LOCATION '/tmp/twits'

 * hive://user1@master.ykono.work:10000
Done.


[]

In [26]:
%%sql
select count(*) from twits

 * hive://user1@master.ykono.work:10000
Done.


_c0
294


In [18]:
%%sql
select * from twits limit 3

 * hive://user1@master.ykono.work:10000
Done.


messages


データ変換のためのテーブルを作成します。

In [27]:
%sql create table message_extracted (symbols array<struct<symbol:string>>, sentiment STRING, body STRING) STORED AS TEXTFILE
%sql create table message_filtered (symbols array<struct<symbol:string>>, sentiment STRING, body STRING) STORED AS TEXTFILE
%sql create table message_exploded (symbol string, sentiment STRING, body STRING) STORED AS TEXTFILE
%sql create table sentiment_data (sentiment int, body STRING) STORED AS TEXTFILE

 * hive://user1@master.ykono.work:10000
Done.
 * hive://user1@master.ykono.work:10000
Done.
 * hive://user1@master.ykono.work:10000
Done.
 * hive://user1@master.ykono.work:10000
Done.


[]

元のデータから必要なデータのみを抽出します。

In [29]:
%%sql
insert overwrite table message_extracted 
select message.symbols, message.entities.sentiment, message.body from twits 
lateral view explode(messages) messages as message

 * hive://user1@master.ykono.work:10000
Done.


[]

In [30]:
%%sql
select * from message_extracted limit 5

 * hive://user1@master.ykono.work:10000
Done.


symbols,sentiment,body
"[{""symbol"":""AAPL""},{""symbol"":""AMZN""},{""symbol"":""CMCSA""},{""symbol"":""DIS""},{""symbol"":""VIAC""}]",None,It’s ‘Star Trek’ vs. ‘Star Wars’ in European Streaming Battle $CMCSA $VIAC $AAPL $AMZN $DIS
[],None,None
"[{""symbol"":""https://newsfilter.io/a/0860a502f00979509c755ec03401ed14""}]",None,None
"[{""symbol"":""AAPL""},{""symbol"":""JD""}]",None,$JD $AAPL
"[{""symbol"":""AAPL""},{""symbol"":""AMRN""},{""symbol"":""SPY""},{""symbol"":""CLVS""}]",None,$SPY $AAPL $CLVS $AMRN


In [31]:
%%sql
select count(*) from message_extracted

 * hive://user1@master.ykono.work:10000
Done.


_c0
10436


データから、メッセージ・ボディが含まれているデータのみを取り出します。同時に、銘柄に対するセンチメントを文字列からを数値に置換します。

In [33]:
%%sql
insert overwrite table message_filtered 
select symbols, 
    case sentiment when 'Bearish' then -2 when 'Bullish' then 2 ELSE 0 END as sentiment, 
    body from message_extracted 
    where body is not null

 * hive://user1@master.ykono.work:10000
Done.


[]

In [34]:
%%sql
select * from message_filtered limit 3

 * hive://user1@master.ykono.work:10000
Done.


symbols,sentiment,body
"[{""symbol"":""AAPL""},{""symbol"":""AMZN""},{""symbol"":""CMCSA""},{""symbol"":""DIS""},{""symbol"":""VIAC""}]",0,It’s ‘Star Trek’ vs. ‘Star Wars’ in European Streaming Battle $CMCSA $VIAC $AAPL $AMZN $DIS
"[{""symbol"":""AAPL""},{""symbol"":""JD""}]",0,$JD $AAPL
"[{""symbol"":""AAPL""},{""symbol"":""AMRN""},{""symbol"":""SPY""},{""symbol"":""CLVS""}]",0,$SPY $AAPL $CLVS $AMRN


一つのメッセージに複数の銘柄が紐づけられています。データ正規化のため、データ１行につき、一つの銘柄を持つようにデータを変換します（同じメッセージを持つ行が複数作られます）。

In [36]:
%%sql
insert overwrite table message_exploded 
select symbol.symbol, sentiment, body from message_filtered lateral view explode(symbols) symbols as symbol

 * hive://user1@master.ykono.work:10000
Done.


[]

In [37]:
%%sql
select * from message_exploded limit 3

 * hive://user1@master.ykono.work:10000
Done.


symbol,sentiment,body
AAPL,0,It’s ‘Star Trek’ vs. ‘Star Wars’ in European Streaming Battle $CMCSA $VIAC $AAPL $AMZN $DIS
AMZN,0,It’s ‘Star Trek’ vs. ‘Star Wars’ in European Streaming Battle $CMCSA $VIAC $AAPL $AMZN $DIS
CMCSA,0,It’s ‘Star Trek’ vs. ‘Star Wars’ in European Streaming Battle $CMCSA $VIAC $AAPL $AMZN $DIS


ここまでの操作で、元の複雑な構造のデータから、１レコードにつき、銘柄、センチメント、メッセージ本文を持つフォーマットに変換されました。
銘柄毎のセンチメントの件数などの分析を行うには、このテーブルを利用します。

この後の感情分析では、メッセージ本文の文字列から、センチメントを判定する予測モデルを構築します。そのため銘柄情報は利用しないため、センチメントとメッセージ本文のみを取り出します。

In [38]:
%%sql
insert overwrite table sentiment_data 
select sentiment, body from message_filtered

 * hive://user1@master.ykono.work:10000
Done.


[]

In [39]:
%%sql
select * from sentiment_data limit 10

 * hive://user1@master.ykono.work:10000
Done.


sentiment,body
0,It’s ‘Star Trek’ vs. ‘Star Wars’ in European Streaming Battle $CMCSA $VIAC $AAPL $AMZN $DIS
0,$JD $AAPL
0,$SPY $AAPL $CLVS $AMRN
2,$AAPL Repo 8AM tomorrow
-2,$AAPL bought 320 puts at the hod should open nicely looking for 70-80%+ at open
-2,$AAPL VIRUS GANG
-2,$SPY think of the last person in china who touched your new iphone before you open the box
0,$SPY $AMD $FB $BABA $AAPL By Morning everyone will be like
0,Apple Reports Earnings Next Week: Here’s How to Tell if Apple Will Meet Expectations $AAPL @apple #apple #earnings http://bit.ly/2RigUf2
2,$AAPL $335 before ER


### JSONファイルの作成

加工したデータをJSONファイルとして出力します。

感情分析を担当するデータサイエンティスト・機械学習エンジニアは、このJSONファイルを使います。

In [40]:
%sql DROP TABLE IF EXISTS json_message
%sql create table json_message (message STRING) STORED AS TEXTFILE

 * hive://user1@master.ykono.work:10000
Done.
 * hive://user1@master.ykono.work:10000
Done.


[]

In [41]:
%%sql
insert overwrite table json_message
select to_json(named_struct('message_body', body, 'sentiment', sentiment)) from sentiment_data

 * hive://user1@master.ykono.work:10000
Done.


[]

In [42]:
%%sql
select * from json_message limit 5

 * hive://user1@master.ykono.work:10000
Done.


message
"{""message_body"":""It’s ‘Star Trek’ vs. ‘Star Wars’ in European Streaming Battle $CMCSA $VIAC $AAPL $AMZN $DIS "",""sentiment"":0}"
"{""message_body"":""$JD $AAPL"",""sentiment"":0}"
"{""message_body"":""$SPY $AAPL $CLVS $AMRN "",""sentiment"":0}"
"{""message_body"":""$AAPL Repo 8AM tomorrow"",""sentiment"":2}"
"{""message_body"":""$AAPL bought 320 puts at the hod should open nicely looking for 70-80%+ at open"",""sentiment"":-2}"


**`HQL_SELECT_MESSAGE`をあなたが作成したデータベースを指定してください**

In [45]:
#from __future__ import print_function

HQL_SELECT_MESSAGE = "select * from default.json_message"

spark = SparkSession\
    .builder\
    .appName("JsonGen")\
    .getOrCreate()
    
spark.sparkContext.setLogLevel("ERROR")

json_list = spark.sql(HQL_SELECT_MESSAGE)

path = "./output_1_0212.json"

with open(path, mode='w') as f:
    f.write('{"data":[')
    bool_first_line = True
    for row in json_list.rdd.collect():
        if bool_first_line:
            bool_first_line = False
            f.write(row.message)
        else:
            # あまりスマートではありませんが、ある程度の量のデータを使ったDeep Learning処理をシミュレーションするため、
            # 同じ情報を使って、データを嵩増ししています。
            # API利用の制約や、演習時間の制約がなければ、
            # 上記のWebスクレイピングで、大量の訓練データを取得することが可能です。
            #for i in range(100): 
            for i in range(1):
                f.write(",\n")
                f.write(row.message)
    
    f.write("]}")

In [47]:
!ls -l | grep output

-rw-r--r-- 1 cdsw cdsw   1157780 Feb 12 08:36 output_1_0212.json
-rwx------ 1 cdsw cdsw 115763447 Feb 12 08:35 output.json


In [48]:
!head output_1_0212.json
!tail output_1_0212.json

{"data":[{"message_body":"It’s ‘Star Trek’ vs. ‘Star Wars’ in European Streaming Battle  $CMCSA $VIAC $AAPL $AMZN $DIS ","sentiment":0},
{"message_body":"$JD $AAPL","sentiment":0},
{"message_body":"$SPY $AAPL $CLVS $AMRN ","sentiment":0},
{"message_body":"$AAPL Repo 8AM tomorrow","sentiment":2},
{"message_body":"$AAPL bought 320 puts at the hod should open nicely looking for 70-80%+ at open","sentiment":-2},
{"message_body":"$AAPL VIRUS GANG","sentiment":-2},
{"message_body":"$SPY think of the last person in china who touched your new iphone before you open the box","sentiment":-2},
{"message_body":"$SPY $AMD $FB $BABA $AAPL By Morning everyone will be like","sentiment":0},
{"message_body":"Apple Reports Earnings Next Week: Here’s How to Tell if Apple Will Meet Expectations $AAPL @apple #apple #earnings http://bit.ly/2RigUf2","sentiment":0},
{"message_body":"$AAPL $335 before ER","sentiment":2},
{"message_body":"$YUMA and $SAEX could move this week if oil prices recover 👀🎣⏱","sentiment

JSONフォーマットでのデータ読み込み

In [51]:
with open('./output_1_0212.json', 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': 'It’s ‘Star Trek’ vs. ‘Star Wars’ in European Streaming Battle  $CMCSA $VIAC $AAPL $AMZN $DIS ', 'sentiment': 0}, {'message_body': '$JD $AAPL', 'sentiment': 0}, {'message_body': '$SPY $AAPL $CLVS $AMRN ', 'sentiment': 0}, {'message_body': '$AAPL Repo 8AM tomorrow', 'sentiment': 2}, {'message_body': '$AAPL bought 320 puts at the hod should open nicely looking for 70-80%+ at open', 'sentiment': -2}, {'message_body': '$AAPL VIRUS GANG', 'sentiment': -2}, {'message_body': '$SPY think of the last person in china who touched your new iphone before you open the box', 'sentiment': -2}, {'message_body': '$SPY $AMD $FB $BABA $AAPL By Morning everyone will be like', 'sentiment': 0}, {'message_body': 'Apple Reports Earnings Next Week: Here’s How to Tell if Apple Will Meet Expectations $AAPL @apple #apple #earnings http://bit.ly/2RigUf2', 'sentiment': 0}, {'message_body': '$AAPL $335 before ER', 'sentiment': 2}]


データ件数の確認

In [52]:
print(len(twits['data']))

7260


### 最後に

データベースを削除する場合は、**データベース名を適切に変更した後で**下記を実行します。

In [30]:
%sql DROP DATABASE IF EXISTS admin CASCADE

 * hive://admin@master.ykono.work:10000
Done.


[]